In [6]:
import numpy as np
import pandas as pd
import hicstraw
from matplotlib import pyplot as plt

In [7]:
hic_path = "/media/jordan990301/Samsung_T5/HiC_Datasets/Rao_2014/GM12878/MAPQGE30/GSE63525_GM12878_insitu_primary_replicate_combined_30.hic"
input_path = "/home/jordan990301/PCA_Experiments/outputs/GM12878/100Kb/origin"

In [8]:
hic = hicstraw.HiCFile(hic_path)
chrom_list = []

for chrom in hic.getChromosomes():
    if (chrom.name != "All" and chrom.name != "MT"):
        chrom_list.append(chrom.name)

print(chrom_list)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']


In [9]:
for chrom in chrom_list:
    # Read in the Pearson correlatin matrix
    pearson_df = pd.read_table(f"{input_path}/origin_pearson_chrom{chrom}.txt", header=None, sep=" ")
    pearson_df.pop(pearson_df.columns[-1])
    pearson_df = pearson_df.dropna(axis=0, how="all").reset_index(drop=True)
    pearson_df = pearson_df.dropna(axis=1, how="all")
    # These code are required since dropping the columns in pandas can't be renamed to index directly. 
    numpy_pearson_df = pearson_df.values
    pearson_df = pd.DataFrame(numpy_pearson_df)
    pearson_df = pd.DataFrame(pearson_df[0])

    # Read in the Eigenvector 1
    EV1_df = pd.read_table(f"{input_path}/origin_pc1_chrom{chrom}.txt", header=None, sep=" ")
    EV1_df = EV1_df.dropna(axis=0, how="all").reset_index(drop=True)

    if (len(pearson_df) == len(EV1_df)):
        # Compare the difference between the sign of these two signals.
        Corr1_Pos = pearson_df > 0
        EV1_Pos = EV1_df > 0
        
        # Turn into python list
        Corr1_Pos = list(Corr1_Pos[0].values)
        EV1_Pos = list(EV1_Pos[0].values)
        
        Difference_Count = 0
        for i in range(len(Corr1_Pos)):
            if(Corr1_Pos[i] != EV1_Pos[i]):
                Difference_Count += 1

        # output_log_path = "/home/jordan990301/PCA_Experiments/outputs/Logs/EV1-Corr1/100Kb"
        # with open(f'{output_log_path}/GM12878_Chr{chrom}_EV1_CorrEntry1_diff.txt',"w+") as f:
        #     f.write("Pearson_Entry1_Pos entry num: ")
        #     f.write(str(len(Pearson_Entry1_Pos)))

        #     f.write("\n")
        #     f.write("EV1_Pos entry num: ")
        #     f.write(str(len(EV1_Pos)))

        #     f.write("\n")
        #     f.write("Difference_Count: ")
        #     f.write(str(Difference_Count))
        
        # Predict the EV1 distribution according to the CorrEntry1 and it's corresponding sign of EV1. 
        Pos_Count = 0 
        Neg_Count = 0
        for i in range(len(Corr1_Pos)):
            if (Corr1_Pos[i]):
                if(EV1_Pos[i]): Pos_Count += 1
                else: Neg_Count += 1

        EV1_Colors = ['b' if i else 'r' for i in EV1_Pos]
        Corr1_Viz = [1 if i else -1 for i in Corr1_Pos]
        EV1_Pred = []

        if (Pos_Count > Neg_Count):
            EV1_Pred = [1 if i else -1 for i in Corr1_Pos]
        else:
            EV1_Pred = [-1 if i else 1 for i in Corr1_Pos]

        # Visualization
        x = [i + 1 for i in range(len(Corr1_Pos))]
        
        output_plot_path = "/home/jordan990301/PCA_Experiments/outputs/plots/EV1-EV1_Pred/100Kb"
        plt.xticks(np.arange(0, len(Corr1_Pos), 50)) 
        plt.rcParams["figure.figsize"] = [20, 5]
        plt.rcParams["figure.autolayout"] = True
        plt.scatter(x, EV1_Pred, c=EV1_Colors)
        plt.savefig(f'{output_plot_path}/GM12878_Chr{chrom}_100Kb.png')
        plt.clf() 
        
        output_plot_path = "/home/jordan990301/PCA_Experiments/outputs/plots/EV1-Corr1/100Kb"
        plt.xticks(np.arange(0, len(Corr1_Pos), 50)) 
        plt.rcParams["figure.figsize"] = [20, 5]
        plt.rcParams["figure.autolayout"] = True
        plt.scatter(x, Corr1_Viz, c=EV1_Colors)
        plt.savefig(f'{output_plot_path}/GM12878_Chr{chrom}_100Kb.png')
        plt.clf() 

<Figure size 2000x500 with 0 Axes>